# Dask Parallel Computing Demo

This notebook demonstrates how to use Dask for parallel computing in Python. We'll compare sequential execution with Dask's delayed execution to show performance improvements through parallelization.

## What is Dask?

Dask is a flexible library for parallel computing in Python that scales Python workflows from single machines to large clusters. It provides:
- **Delayed execution**: Build computation graphs and execute them efficiently
- **Parallel collections**: Arrays, DataFrames, and Bags that work in parallel
- **Task scheduling**: Intelligent task scheduling for optimal performance

## Import Required Libraries

Let's start by importing the necessary libraries:
- `dask`: For parallel computing capabilities
- `time`: To simulate computational delays in our examples

In [ ]:
import dask
import time

## Define Helper Functions

We'll create two simple functions that include artificial delays using `time.sleep(1)` to simulate computationally expensive operations:

- `square(n)`: Returns the square of a number
- `add(m, n)`: Returns the sum of two numbers

The 1-second delay in each function will help us see the performance difference between sequential and parallel execution.

In [ ]:
def square(n):
    time.sleep(1)
    return n * n
    
def add(m, n):
    time.sleep(1)
    return m + n

## Sequential Execution Example

First, let's run our functions sequentially (one after another) and measure the time:

1. Calculate `square(1)` - takes ~1 second
2. Calculate `square(2)` - takes ~1 second  
3. Add the results together - takes ~1 second

**Expected total time: ~3 seconds**

In [ ]:
%%time 

x = square(1)
y = square(2)
z = add(x, y)

## Dask Delayed Execution

Now let's use Dask's `delayed` decorator to create a computation graph instead of executing immediately:

- `dask.delayed()` wraps our functions to create lazy computation tasks
- No computation happens yet - we're just building a graph of dependencies
- The two `square()` operations can potentially run in parallel since they don't depend on each other

In [ ]:
x = dask.delayed(square)(1)
y = dask.delayed(square)(2)
z = dask.delayed(add)(x, y)

## Visualize the Computation Graph (Optional)

Dask can visualize the computation graph to show task dependencies. Uncomment the line below to see how Dask plans to execute our tasks:

- You'll see that `square(1)` and `square(2)` can run in parallel
- The `add` operation depends on both square operations completing first

In [ ]:
#z.visualize(rankdir='LR')

## Execute the Dask Computation

Now let's trigger the actual computation using `.compute()` and measure the execution time:

**Expected improvement**: Since `square(1)` and `square(2)` can run in parallel, the total time should be closer to ~2 seconds instead of 3 seconds.

In [ ]:
%%time
z.compute()

## Scaling Up: Sequential Processing of 100 Items

Let's see the performance difference with a larger workload. Here we'll process 100 numbers sequentially:

- Each `square(i)` operation takes 1 second
- With 100 operations running sequentially, this should take approximately **100 seconds**
- Plus the time to sum all results

In [ ]:
%%time

x = [square(i) for i in range(100)]
y = sum(x)

## Scaling Up: Parallel Processing with Dask

Now let's create the same computation using Dask delayed operations:

- We create 100 delayed square operations
- All of these can potentially run in parallel (limited by available CPU cores)
- The final sum operation waits for all square operations to complete

In [ ]:
x = [dask.delayed(square)(i) for i in range(100)]
y = dask.delayed(sum)(x)

In [ ]:
%%time

y.compute()

## Key Takeaways

This demo illustrates several important concepts about parallel computing with Dask:

### 1. **Lazy Evaluation**
- Dask builds a computation graph before executing anything
- This allows for optimization and parallel execution planning

### 2. **Automatic Parallelization**
- Independent tasks can run simultaneously across multiple CPU cores
- Dask handles task scheduling and resource management automatically

### 3. **Scalability Benefits**
- Small tasks (like our 3-operation example) may not show dramatic speedups due to overhead
- Larger workloads (like our 100-operation example) demonstrate significant performance gains
- Real-world data processing tasks with thousands or millions of operations see even greater benefits

### 4. **Easy Integration**
- Minimal code changes required to parallelize existing Python workflows
- Works seamlessly with NumPy, Pandas, and other scientific Python libraries